In [1]:
import os
import cv2
import numpy as np
from pathlib import Path

import sys
sys.path.append('../src')  # Add path to import preprocess module
from preprocess import preprocess_image, adjust_boxes, plot_original_and_preprocessed_images


# Preprocess

In [2]:
# Input paths (only training data)
image_dir = '../data/train/images'
label_dir = '../data/train/labels'

# Output paths (outside data folder)
output_image_dir = '../enhance_image/images'
output_label_dir = '../enhance_image/labels'

In [3]:
# Create output directories
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Supported extensions
image_extensions = ('.png', '.jpg', '.jpeg')

In [4]:
# List all images
image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(image_extensions)])


In [5]:
# Helper: read YOLO label
def read_yolo_label(file_path):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, cx, cy, w, h = map(float, parts)
                x1 = (cx - w/2) * 416
                y1 = (cy - h/2) * 416
                x2 = (cx + w/2) * 416
                y2 = (cy + h/2) * 416
                boxes.append([x1, y1, x2, y2, int(cls)])
    return boxes

In [6]:
# Helper: write YOLO label
def write_yolo_label(file_path, boxes):
    with open(file_path, 'w') as f:
        for box in boxes:
            x1, y1, x2, y2, cls = box
            cx = ((x1 + x2) / 2) / 416
            cy = ((y1 + y2) / 2) / 416
            w = (x2 - x1) / 416
            h = (y2 - y1) / 416
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")


In [7]:
# Get all image files
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

for img_name in sorted(image_files):
    base_name = os.path.splitext(img_name)[0]
    img_path = os.path.join(image_dir, img_name)
    label_path = os.path.join(label_dir, base_name + ".txt")

    if not os.path.exists(label_path):
        print(f"❌ Skipped (no matching label): {img_name}")
        continue

    # Load image and preprocess
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read image: {img_path}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pre_img, original_size = preprocess_image(img)

    # Read and adjust bounding boxes
    boxes = read_yolo_label(label_path)
    new_boxes = adjust_boxes(boxes, original_size)

    # Save preprocessed image (same name)
    save_img_path = os.path.join(output_image_dir, img_name)
    cv2.imwrite(save_img_path, (pre_img * 255).astype(np.uint8)[:, :, ::-1])  # RGB to BGR

    # Save label (same name)
    save_label_path = os.path.join(output_label_dir, base_name + ".txt")
    write_yolo_label(save_label_path, new_boxes)

    print(f"✅ Processed & saved: {img_name}")

✅ Processed & saved: 00000020_jpg.rf.2d8902d95307cb9a1bb38701827f2013.jpg
✅ Processed & saved: 00000020_jpg.rf.442a8eff7ef8e3b94e35395b9e193b76.jpg
✅ Processed & saved: 00000020_jpg.rf.a533885689303c9fae6d623f2134f0d9.jpg
✅ Processed & saved: 00000039_jpg.rf.a8aa969024f6b93f1854b32e9ff91396.jpg
✅ Processed & saved: 00000039_jpg.rf.bc8f24dabb26c9ff5b6210cd58b610c6.jpg
✅ Processed & saved: 00000039_jpg.rf.e81bde37659bba93521fcef56bdc2301.jpg
✅ Processed & saved: 00000084_jpg.rf.43ae71bf84505951a7e6803a422cdeb8.jpg
✅ Processed & saved: 00000084_jpg.rf.7add194c5f3111742504d00292335447.jpg
✅ Processed & saved: 00000084_jpg.rf.ff39553112e8260e3534643c4103685d.jpg
✅ Processed & saved: 00000212_jpg.rf.8119a39c121f9abe13085e469471ef08.jpg
✅ Processed & saved: 00000212_jpg.rf.8592055c010f5e5d77d88d5d8cd8f8e5.jpg
✅ Processed & saved: 00000212_jpg.rf.cb055f28ed4022de6ad98027162d685d.jpg
✅ Processed & saved: 00000246_jpg.rf.086b4c7878344dd590d77f79c21ab368.jpg
✅ Processed & saved: 00000246_jpg.rf.b